This notebook extracts the lab data and reformat them to have unified denomination. Then extract labels of interest: death and length of stay.

This code relies on [`MIMIC_extract`](https://github.com/MLforHealth/MIMIC_Extract) matching dictionary and the [MIMIC III](https://physionet.org/content/mimiciii/1.4/) data.
To begin, download the data and update the following variable `PATH`.

In [ ]:
PATH = '/home/vincent/Desktop/Cambridge/Data/Mimic/' #'/home/vjemj2/rds/hpc-work/data/mimic/'

In [ ]:
import numpy as np
import pandas as pd

# Lab data

### Mapping labs variables

In [ ]:
mapping = pd.read_csv(PATH + 'itemid_to_variable_map.csv', index_col = 'ITEMID', dtype = {'ITEMID': int})

In [ ]:
mapping

In [ ]:
mapping = pd.read_csv(PATH + 'itemid_to_variable_map.csv', index_col = 'ITEMID', dtype = {'ITEMID': int})
mapping = mapping[(mapping['LEVEL2'] != '') &\
                  (mapping['COUNT'] > 0) &\
                  (mapping['STATUS'] == 'ready')
                 ]

In [ ]:
mapping.head()

### Extract lab data

In [ ]:
labs = pd.read_csv(PATH + 'LABEVENTS.csv', parse_dates = ['CHARTTIME'])

In [ ]:
# Select data and replace itemid with standard format
labs = labs[labs.ITEMID.isin(mapping.index)][['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']]
labs['Lab'] = mapping['LEVEL1'].loc[labs['ITEMID']].values

In [ ]:
labs.head()

# Labels

Read the patients' outcome and extract the temporal information for labelling the data

In [ ]:
genderAge = pd.read_csv(PATH + 'PATIENTS.csv', usecols = ['SUBJECT_ID', 'GENDER', 'DOB'], parse_dates = ['DOB'])

In [ ]:
admissions = pd.read_csv(PATH + 'ADMISSIONS.csv',  
                         usecols = ['SUBJECT_ID', 'HADM_ID', 'ADMISSION_TYPE', 'HOSPITAL_EXPIRE_FLAG', 'DISCHARGE_LOCATION', 
                                    'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'INSURANCE', 'DIAGNOSIS'],
                         parse_dates = ['ADMITTIME', 'DISCHTIME', 'DEATHTIME'])
admissions = admissions.merge(genderAge, on = 'SUBJECT_ID')

In [ ]:
# Focus only on adults - Update 12.10.2021
removed_nan = admissions[['ADMITTIME', 'DOB']].dropna()
admissions['AGE'] = np.nan
admissions['AGE'][removed_nan.index] = [date.days for date in (removed_nan.ADMITTIME.dt.to_pydatetime() - removed_nan.DOB.dt.to_pydatetime())]
admissions.AGE /= 365

admissions = admissions[admissions.AGE > 18]

In [ ]:
# Focus on last visits (as space between visit might change process)
admissions = admissions.loc[admissions['SUBJECT_ID'].drop_duplicates('last').index]

In [ ]:
# Change times to hours since admission to the hospital
# Update to death in Hospital 3.8.2023
admissions['Death'] = admissions['DEATHTIME'] - admissions['ADMITTIME']
admissions['Stay'] = admissions['DISCHTIME'] - admissions['ADMITTIME']

In [ ]:
# Remove incoherent dates
# Revision 12.7.2023
admissions = admissions[((admissions['Death'].dt.total_seconds().fillna(1) > 0)) & ((admissions['Stay'].dt.total_seconds().fillna(1) > 0))]

In [ ]:
# Add temproral information
# Shift of 4 hours to have patients of weekend from 8pm on Friday to 8pm on Sunday to ensure patients spends at least 12 hours in a low attention environement # Correction => 6.9.2023
admissions['Day'] = (admissions['ADMITTIME'] + pd.to_timedelta('4 hours')).dt.weekday

In [ ]:
# Create index for easier search
admissions = admissions.set_index('SUBJECT_ID')
admissions.head()

In [ ]:
assert len(admissions.HADM_ID.unique()) == len(admissions), \
    "Different patients have the same HADM_ID, might be a problem for the rest of the code"

# Transformation labs

In [ ]:
# Remove unecessary HADM_ID
labs = labs[labs.HADM_ID.isin(admissions.HADM_ID)]

In [ ]:
# Change time event to time since admission
# Update time after admission to the ICU
labs['Time'] = labs.CHARTTIME.values - admissions.ADMITTIME.loc[labs.SUBJECT_ID].values

In [ ]:
labs.head()

# Reformat

Clean reformating of the dataframe for saving

In [ ]:
labs.rename(columns = {"SUBJECT_ID": "Patient",
                       "VALUENUM": "Value"}, inplace = True)
labs = labs.reset_index(drop = True)[['Patient', 'Time', 'Lab', 'Value']]
labs.head()

In [ ]:
admissions.rename_axis(index = "Patient", inplace = True)
admissions.head()

# Selection First days

In [ ]:
first_day = pd.to_timedelta('1 days')
admissions = admissions[(admissions.Stay >= first_day) &\
                        ((admissions.Death >= admissions.Stay) | admissions.Death.isna())] # Stayed in hospital at least 24 hours - Time of prediction (and match information)
admissions
labs = labs[(labs.Time < first_day) &\
              labs.Patient.isin(admissions.index)] # Select all data relevant

# Cleaning labs

In [ ]:
# Remove duplicates: same test multiple time at the same time
labs = labs[~labs.set_index(['Patient', 'Time', 'Lab']).index.duplicated(keep = False)]

In [ ]:
# Pivot to have test as columns
labs = labs.pivot(index = ['Patient', 'Time'], columns = 'Lab')

# Change index to have days
labs.index = pd.MultiIndex.from_tuples([(x[0], x[1].total_seconds()/(3600.*24)) for x in labs.index], names = labs.index.names)

In [ ]:
# Keep labs that at least 5% population has one
# New subselection => 12.7.2023
labs = labs[labs.columns[(labs.groupby('Patient').count() >= 1).mean() > 0.05]]
labs.head()

In [ ]:
# Keep labs only after admission
# Justification: medical process prior to admission might be really different
# New subselection => 6.7.2021
labs = labs[labs.index.get_level_values('Time') >= 0]
labs.head()

In [ ]:
# Remove multi-level index -> 21.12.2022
labs.columns = labs.columns.droplevel() 

In [ ]:
# Remove empty lines
labs = labs.dropna(how = 'all')

In [ ]:
# Keep patients that have at least two measurements
patients = labs.groupby('Patient').size() > 1
patients = patients[patients].index
labs = labs[labs.index.get_level_values('Patient').isin(patients)]

In [ ]:
print("Average number of test performed : {:.2f}".format(labs.groupby('Patient').size().mean()))

In [ ]:
admissions

# Clean outcomes

In [ ]:
# Remove patients with no labs
admissions = admissions.loc[labs.index.get_level_values(0).unique()]

In [ ]:
admissions['Time'] = admissions.Death.fillna(admissions.Stay) # Either death or length of stay
admissions['Time'] = admissions['Time'].dt.total_seconds() / (24 * 60 * 60)
admissions['Event'] = admissions['HOSPITAL_EXPIRE_FLAG'] == 1 # Predict Death in Hospital otherwise dischaged (censored if == 0)

admissions['Remaining'] = (admissions.Time.loc[labs.index.get_level_values(0)] - labs.index.get_level_values(1)).groupby('Patient').last()

# Save

Rename columns and save all the data and labels

In [ ]:
labs.to_csv('data/mimic/labs_first_day.csv')
admissions.to_csv('data/mimic/outcomes_first_day.csv')